In [1]:
from config import int_key
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [28]:
#Get only weekdays in an array so we can search the News API
Tickers= ["AAPL", "BAC", "VZ", "WMT", "XOM", "AMZN",]
stock = requests.get(f"https://api.iextrading.com/1.0/stock/{Tickers[0]}/chart/1m")
stocks = stock.json()
dates = []

for i in range(len(stocks)):
    dates.append(stocks[i]["date"])

    


In [18]:
article_headline = []
date_published = []
body = []
import aylien_news_api
from aylien_news_api.rest import ApiException

# Configure API key authorization: app_id
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = 'abf8f744'
# Configure API key authorization: app_key
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = '2ca6bef1e403bc29c894f40e79feaf14'

# create an instance of the API class
api_instance = aylien_news_api.DefaultApi()
for i in range(len(dates)):
    opts = {
      'body': 'Verizon',
      'title': 'Verizon',
      'language': ['en'],
      'not_language': ['es', 'it'],
      'published_at_start': f"{dates[i]}T01:00:10Z",
      'published_at_end': f"{dates[i]}T23:00:10Z",
      'per_page': 100,
      'entities_body_text': [
        'Verizon']  
      #'cursor': 'AoJw6uq3sOgCL1N0b3J5IDIyMjEwMTA4OQ=='

    }

    try:
        # List stories
        api_response = api_instance.list_stories(**opts)

    #     print("API called successfully. Returned data: ")
    #     print("========================================")
        for story in api_response.stories:

            article_headline.append(story.title)
            date_published.append(story.published_at.date())
            body.append(story.body)

    except ApiException as e:
        print("Exception when calling DefaultApi->list_stories: %sn" % e)


In [19]:
len(article_headline)

974

In [20]:
article_sentiment = []
article_date = []
for i in range(len(article_headline)):
    #Request the sentiment
    try:
        url = f"http://api.intellexer.com/analyzeSentiments?apikey={int_key}"
        sentiment = requests.post(url, data= json.dumps([{"id": "1", "text": article_headline[i] + body[i]}]),headers = {"Content-Type": "application/json", "Cache-Control": "no-cache"}).json()
        article_date.append(date_published[i])
        article_sentiment.append(float(sentiment['sentiments'][0]['w']))
    except:
        continue

In [21]:
len(article_sentiment)

970

In [22]:
company_cols = {"Date": article_date,
                "Sentiment": article_sentiment}
company_data = pd.DataFrame(company_cols)
#export to csv when ready
company_data.to_csv(f"Sentiments/{company}_sentiment_final.csv)

In [ ]:
#Create a function to print out ticker prices for the dates
def get_stock_csv(ticker):
    change = []
    stock_date = []
    date = []
    stock = requests.get(f"https://api.iextrading.com/1.0/stock/{ticker}/chart/1m")
    stocks = stock.json()
    
    for i in range(len(stocks)):
        change.append(stocks[i]['changePercent'])
        stock_date.append(stocks[i]['date'])
    stocks_df = pd.DataFrame({"Date":stock_date, "Percent Change": change})
    stocks_df.to_csv(f"Stock Prices/{ticker}.csv")
    
    
 #Print out the company prices
for ticker in Tickers:
    get_stock_csv(ticker)  
